In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import pennylane as qml

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from torch.utils.data import Dataset, DataLoader

## Dataset

In [5]:
df = pd.read_csv('./jena_climate_2009_2016.csv')

In [6]:
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [7]:
train_fraction = int(0.745 * df.shape[0])

In [8]:
def Normalize(data, split):
    mean = data[:split].mean(axis=0)
    std = data[:split].std(axis=0)
    return (data - mean) / std # normalize the dataset 

In [9]:
features = df[[df.columns[i+1] for i in [0, 1, 5, 7, 8, 10, 11]]]
features.index = df["Date Time"]
features.head()

,p (mbar),T (degC),VPmax (mbar),VPdef (mbar),sh (g/kg),rho (g/m**3),wv (m/s)
Date Time,,,,,,,
01.01.2009 00:10:00,996.52,-8.02,3.33,0.22,1.94,1307.75,1.03
01.01.2009 00:20:00,996.57,-8.41,3.23,0.21,1.89,1309.80,0.72
01.01.2009 00:30:00,996.53,-8.51,3.21,0.20,1.88,1310.24,0.19
01.01.2009 00:40:00,996.51,-8.31,3.26,0.19,1.92,1309.19,0.34
01.01.2009 00:50:00,996.51,-8.27,3.27,0.19,1.92,1309.00,0.32


In [10]:
features = Normalize(features.values, train_fraction)
features = pd.DataFrame(features)
features.head()

,0,1,2,3,4,5,6
0,0.955617,-2.018504,-1.327546,-0.780846,-1.517837,2.257331,-0.724686
1,0.961724,-2.064146,-1.340719,-0.782953,-1.536565,2.308011,-0.927967
2,0.956838,-2.075849,-1.343353,-0.785060,-1.540311,2.318888,-1.275511
3,0.954395,-2.052443,-1.336767,-0.787167,-1.525328,2.292930,-1.177150
4,0.954395,-2.047762,-1.335450,-0.787167,-1.525328,2.288233,-1.190265


In [11]:
X_train = features.loc[:train_fraction-1]
X_valid = features.loc[train_fraction:]

X_train = X_train[range(7)].values
y_train = np.array(features.iloc[792: 792 + train_fraction][[1]])

### Validation Dataset

In [110]:
valid_end = len(X_valid) - 792
valid_start = train_fraction + 792

In [111]:
X_valid = X_valid.iloc[:valid_end][range(7)].values
y_valid = np.array(features.iloc[valid_start:][[1]])

### PyTorch Dataset

In [12]:
class TimeseriesDataset(Dataset):
    def __init__(self, X, y, sequence_length: int=240, sampling_rate: int=6):
        self.X = torch.Tensor(X)
        self.y = torch.Tensor(y)
        self.sequence_length = sequence_length
        self.sampling_rate = sampling_rate
        self.indices = [i for i in range(0, self.X.shape[0] - self.sequence_length, self.sampling_rate)]

    def __len__(self) -> int:
        return len(self.indices)

    def __getitem__(self, idx: int):
        start_idx = self.indices[idx]
        end_idx = start_idx + self.sequence_length
        return self.X[start_idx:end_idx], self.y[end_idx-1]

In [13]:
WeatherDataset = TimeseriesDataset(X_train, y_train)

In [14]:
print(f"Length of dataset, {len(WeatherDataset)}")

Length of dataset, 52179


In [15]:
WeatherLoader = DataLoader(WeatherDataset, batch_size=256, shuffle=False)

In [16]:
for batch in WeatherLoader:
    X_batch, y_batch = batch
    print(X_batch.shape, y_batch.shape)
    break

torch.Size([256, 240, 7]) torch.Size([256, 1])


In [112]:
ValidDataset = TimeseriesDataset(X_valid, y_valid)

In [113]:
print(f"Length of validation dataset, {len(ValidDataset)}")

Length of validation dataset, 17702


In [114]:
ValidLoader = DataLoader(ValidDataset, batch_size=256, shuffle=False)

## QuLTSF Model


In [98]:
class QuLTSF(nn.Module):
    def __init__(
        self,
        lookback_window_size,
        forecast_window_size,
        n_qubits: int,
        n_layers: int,
    ):
        super(QuLTSF, self).__init__()
        self.lookback_window_size = lookback_window_size
        self.forecast_window_size = forecast_window_size
        self.dev = qml.device('default.qubit', wires=n_qubits)

        @qml.qnode(self.dev, interface='torch', diff_method='best')
        def PQC(inputs, weights):
            qml.AmplitudeEmbedding(features=inputs, wires=range(n_qubits), normalize=True)
            qml.StronglyEntanglingLayers(weights=weights, wires=range(n_qubits))
            return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

        self.qfunc = PQC
        weight_shapes = {"weights" : (n_layers, n_qubits, 3)}
        self.input_layer = nn.Linear(lookback_window_size, 2 ** n_qubits)
        self.q_layer = qml.qnn.TorchLayer(self.qfunc, weight_shapes)
        self.output_layer = nn.Linear(n_qubits, forecast_window_size)

    def forward(self, batch_input):
        x = batch_input.permute(0, 2, 1)
        x = F.relu(
            self.input_layer(x)
        )
        x = self.q_layer(x)
        x = F.relu(
            self.output_layer(
                self.output_layer(x).squeeze()
            )
        )
        return x

In [101]:
Middlefield = QuLTSF(
    lookback_window_size = 240,
    forecast_window_size = 1,
    n_qubits = 7,
    n_layers = 2
)
print(Middlefield)

QuLTSF(
  (input_layer): Linear(in_features=240, out_features=128, bias=True)
  (q_layer): <Quantum Torch Layer: func=PQC>
  (output_layer): Linear(in_features=7, out_features=1, bias=True)
)


## Training the Model

In [102]:
from torch.optim import Adam, lr_scheduler

In [103]:
optimizer = Adam(Middlefield.parameters(), lr=1e-4)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=3)

In [104]:
loss_fn = nn.MSELoss()

In [120]:
for epoch in range(10):
    train_Loss = 0.0
    
    for step, (X_batch, y_batch) in enumerate(WeatherLoader):
        logits = Middlefield(X_batch)
        loss = loss_fn(logits, y_batch)
        Middlefield.zero_grad()
        loss.backward()
        optimizer.step()
        train_Loss += loss.item()

    train_Loss /= (step+1)
    Middlefield.eval()

    val_Loss = 0.0
    with torch.no_grad():
        for step, (X_batch, y_batch) in enumerate(ValidLoader):
            logits = Middlefield(X_batch)
            loss = loss_fn(logits, y_batch)
            val_Loss += loss.item()

    val_Loss /= (step + 1)
    scheduler.step(val_Loss)

    print(f"Epoch [{epoch+1}/{10}], Train Loss: {train_Loss:.4f}, Val Loss: {val_Loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.6f}")

Epoch [1/10], Train Loss: 0.7211, Val Loss: 0.5964, LR: 0.000100
Epoch [2/10], Train Loss: 0.7151, Val Loss: 0.5886, LR: 0.000100
Epoch [3/10], Train Loss: 0.7107, Val Loss: 0.5830, LR: 0.000100
Epoch [4/10], Train Loss: 0.7076, Val Loss: 0.5779, LR: 0.000100
Epoch [5/10], Train Loss: 0.7046, Val Loss: 0.5740, LR: 0.000100
Epoch [6/10], Train Loss: 0.7019, Val Loss: 0.5701, LR: 0.000100
Epoch [7/10], Train Loss: 0.6996, Val Loss: 0.5668, LR: 0.000100
Epoch [8/10], Train Loss: 0.6975, Val Loss: 0.5640, LR: 0.000100
Epoch [9/10], Train Loss: 0.6957, Val Loss: 0.5616, LR: 0.000100
Epoch [10/10], Train Loss: 0.6940, Val Loss: 0.5592, LR: 0.000100
